In [ ]:
import tensorflow as tf
print(tf.__version__)

n_input1 = 28 # pixels of a digit in MNIST
n_input2 = 28 
n_kernel = 3 # CNN with a kernel of 3 by 3 
n_pool = 2 # CNN with a pooling of 2 by 2
n_output = 10 # corresponds to the 10 digits (0-9)
n_batch_size = 50 
n_epochs = 5

# required orderwise tf uses up all the memory of GPU and crashes 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.compat.v1.InteractiveSession(config=config)

# getting the MNIST dataset : 60k of handwritten labelled digits 
    
mnist = tf.keras.datasets.mnist 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], n_input1, n_input2, 1)
x_test = x_test.reshape(x_test.shape[0], n_input1, n_input2, 1)

input_shape = (n_input1, n_input2, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

# Building up the model 

model = tf.keras.models.Sequential() 

model.add(tf.keras.layers.Conv2D(n_input1,kernel_size=(n_kernel,n_kernel), input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(n_pool,n_pool)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2)) # avoid overfitting 
model.add(tf.keras.layers.Dense(n_output, activation=tf.nn.softmax))

# Setting the optimizer and the loss function hyperparameters

model.compile (optimizer='adam',
                loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# training of the model on the GPU

model.fit(x=x_train,y=y_train, batch_size=n_batch_size, epochs=n_epochs)

# evaluation of the model 

model.evaluate(x_test, y_test)